In [2]:
import pandas as pd
final = pd.read_csv(r'final.csv', encoding = 'CP949')
final.head(5)

,시도,시군구,행정번호,년도,우울증환자수,평균연령,총인구수,스트레스인지율,고용률,코로나확진자수,1인당 정신건강예산(원),공공시설개수,우울증지수
0,서울특별시,서울종로구,1111000000,2018,14837,44.2,153065,34.2,59.6,0,5326,44,9.693268
1,서울특별시,서울중구,1114000000,2018,4352,44.6,125725,28.3,59.6,0,5326,42,3.461523
2,서울특별시,서울용산구,1117000000,2018,2370,43.6,228999,20.0,59.6,0,5326,53,1.034939
3,서울특별시,서울성동구,1120000000,2018,3657,42.0,308221,31.3,59.6,0,5326,50,1.186486
4,서울특별시,서울광진구,1121500000,2018,6917,41.4,355559,29.4,59.6,0,5326,47,1.945387


### SVR 하이퍼파라미터 설명

+ kernel   
데이터의 특성을 고차원으로 매핑하여 비선형 관계를 모델링할 때 사용됨  
'linear' (선형)  
'rbf' (방사 기저 함수, 기본값)  
'poly' (다항식) 등  

+ C (기본값 1.0)  
오차에 대한 페널티를 조절하는 매개변수  
작은 C 값은 오차를 허용 / 큰 C 값은 오차를 페널티로 여김  
(C가 크면 모델이 훈련 데이터에 더욱 민감해지고 복잡한 결정 경계를 만듦)

+ gamma  
입력 특성이 얼마나 영향을 받을지를 결정  
작은 gamma 값은 넓은 영역을 의미

In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict

In [13]:
# 독립변수, 종속변수 정의
X = final[['우울증환자수', '평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final['우울증지수']

# 모델 파이프라인 생성 
svr_model = make_pipeline(
    RobustScaler(),
    SVR(kernel='rbf', C = 700, gamma = 0.1)
)

# 모델 성능 검정
rf_cv_scores_r2 = cross_val_score(svr_model, X, y, cv=5, scoring='r2')
rf_cv_scores_mse = cross_val_score(svr_model, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-rf_cv_scores_mse)

print('cv=' , 5, '/ 평균 r2:', round(rf_cv_scores_r2.mean(),4), '/ 표준편차 r2:', round(rf_cv_scores_r2.std(),4), '/ rmse :', round(cv_rmse.mean(),4))
# cv= 5 / 평균 r2: 0.7526 / 표준편차 r2: 0.388 / rmse : 0.458

cv= 5 / 평균 r2: 0.7526 / 표준편차 r2: 0.388 / rmse : 0.458


In [11]:
# 교차 검증 예측 수행
y_pred_cv = cross_val_predict(svr_model, X, y, cv=5)

# R² 계산
r2_cv = r2_score(y, y_pred_cv)

print('predict R²:', round(r2_cv, 4))

predict R²: 0.7256
